# Lab 2.3.5: BERT Fine-tuning for Sentiment Classification

**Module:** 2.3 - Natural Language Processing & Transformers  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Understand what BERT is and how it was pre-trained
- [ ] Load and inspect a pre-trained BERT model
- [ ] Prepare data for fine-tuning
- [ ] Fine-tune BERT for sentiment classification
- [ ] Evaluate model performance and interpret results
- [ ] Leverage DGX Spark's 128GB memory for larger batch sizes

---

## Prerequisites

- Completed: Notebooks 01-04 (Attention, Transformer, Position, Tokenization)
- Knowledge of: PyTorch training loops, classification metrics

---

## Real-World Context

**BERT fine-tuning is used everywhere:**
- **Spam detection** - Is this email spam?
- **Sentiment analysis** - Is this review positive or negative?
- **Intent classification** - What does this customer want?
- **Named Entity Recognition** - Find names, dates, locations
- **Question answering** - Find answers in documents

Transfer learning with BERT typically needs:
- Only a few thousand labeled examples
- Minutes to hours of training (vs weeks from scratch)
- State-of-the-art results on most NLP tasks

---

## ELI5: What is BERT?

> **Imagine a student who has read every book in the library.**
>
> They haven't memorized specific answers, but they deeply understand:
> - How words relate to each other
> - Grammar and sentence structure
> - Context and meaning
>
> **Now you need to teach them to classify movie reviews.**
>
> Because they already understand language, you only need:
> - A few hundred examples of positive/negative reviews
> - A few hours of practice
>
> **That's BERT fine-tuning!**
> - Pre-training: Read lots of text, learn language (done by Google)
> - Fine-tuning: Learn your specific task (you do this)

### BERT's Pre-training Tasks

1. **Masked Language Modeling (MLM)**: Predict missing words
   - Input: "The [MASK] sat on the mat"
   - Predict: "cat"

2. **Next Sentence Prediction (NSP)**: Do sentences follow each other?
   - "The cat sat down." + "It was tired." → Yes (follows)
   - "The cat sat down." + "Stocks rose 5%." → No (random)

---

## Part 1: Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import numpy as np
import matplotlib.pyplot as plt
import time
import gc

# Try to import tqdm for progress bars
try:
    from tqdm.auto import tqdm
    HAS_TQDM = True
except ImportError:
    HAS_TQDM = False
    print("⚠️ tqdm not installed. Progress bars will be disabled.")
    print("Install with: pip install tqdm")
    
    # Create a fallback class that supports set_postfix
    class TqdmFallback:
        """Fallback for tqdm that supports set_postfix."""
        def __init__(self, iterable, **kwargs):
            self.iterable = iterable
            self.desc = kwargs.get('desc', '')
        def __iter__(self):
            return iter(self.iterable)
        def set_postfix(self, *args, **kwargs):
            pass  # No-op for fallback
    
    def tqdm(iterable, **kwargs):
        return TqdmFallback(iterable, **kwargs)

# Hugging Face libraries
try:
    from transformers import (
        BertTokenizer, 
        BertModel, 
        BertForSequenceClassification,
        get_linear_schedule_with_warmup
    )
    from datasets import load_dataset
    HAS_HF = True
except ImportError:
    HAS_HF = False
    print("❌ transformers and/or datasets not installed.")
    print("Install with: pip install transformers datasets")

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU Memory: {gpu_mem:.1f} GB")
    
    # Clear cache before loading large models
    torch.cuda.empty_cache()
    gc.collect()
    
    # DGX Spark optimization: use larger batches based on GPU memory
    if gpu_mem > 100 or "GB10" in torch.cuda.get_device_name(0):
        print("\n✨ DGX Spark detected! Using optimized settings.")
        BATCH_SIZE = 64  # Much larger than typical 16-32
    elif gpu_mem > 40:
        BATCH_SIZE = 32
    elif gpu_mem > 20:
        BATCH_SIZE = 24
    else:
        BATCH_SIZE = 16
else:
    BATCH_SIZE = 8

print(f"\nBatch size: {BATCH_SIZE}")

# Set seed for reproducibility
torch.manual_seed(42)

---

## Part 2: Understanding BERT Architecture

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
bert = BertModel.from_pretrained(model_name)

print(f"Model: {model_name}")
print(f"Vocabulary size: {tokenizer.vocab_size:,}")
print(f"Max sequence length: {tokenizer.model_max_length}")
print(f"\nModel parameters: {sum(p.numel() for p in bert.parameters()):,}")

In [ ]:
# Explore BERT's architecture
print("BERT Architecture:")
print("=" * 50)
print(f"Hidden size: {bert.config.hidden_size}")
print(f"Number of layers: {bert.config.num_hidden_layers}")
print(f"Attention heads: {bert.config.num_attention_heads}")
print(f"FFN dimension: {bert.config.intermediate_size}")
print(f"\nThis matches what we built in notebooks 01-02!")

In [ ]:
# See how BERT processes text
text = "I love this movie! It's absolutely fantastic."

# Tokenize
tokens = tokenizer(text, return_tensors="pt", padding=True)
print("Tokenized input:")
print(f"  Input IDs shape: {tokens['input_ids'].shape}")
print(f"  Tokens: {tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])}")
print(f"  Attention mask: {tokens['attention_mask'][0].tolist()}")

# Forward pass
with torch.no_grad():
    outputs = bert(**tokens)

print(f"\nBERT outputs:")
print(f"  Last hidden state: {outputs.last_hidden_state.shape}")
print(f"  Pooler output:     {outputs.pooler_output.shape}")

### BERT Output Explained:

- **last_hidden_state**: (batch, seq_len, hidden_size) - contextual embeddings for each token
- **pooler_output**: (batch, hidden_size) - the [CLS] token embedding, passed through a dense layer

For classification, we typically use the **pooler_output** as the sentence representation.

---

## Part 3: Load and Prepare Dataset

In [ ]:
# Load IMDB dataset (movie reviews with sentiment labels)
print("Loading IMDB dataset...")
dataset = load_dataset("imdb")

print(f"\nDataset structure:")
print(dataset)

# Look at a sample
print(f"\nSample review (first 300 chars):")
print(f"  Text: {dataset['train'][0]['text'][:300]}...")
print(f"  Label: {dataset['train'][0]['label']} (0=negative, 1=positive)")

In [ ]:
# For faster training, use a subset
# With DGX Spark, we can use more data!
if torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory > 100e9:
    NUM_TRAIN = 5000  # Use more with DGX Spark
    NUM_TEST = 1000
else:
    NUM_TRAIN = 2000
    NUM_TEST = 500

train_dataset = dataset['train'].shuffle(seed=42).select(range(NUM_TRAIN))
test_dataset = dataset['test'].shuffle(seed=42).select(range(NUM_TEST))

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Create a PyTorch Dataset

class IMDBDataset(Dataset):
    """
    Custom Dataset for IMDB reviews.
    
    Handles tokenization and padding.
    """
    
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_ds = IMDBDataset(train_dataset, tokenizer, max_length=256)
test_ds = IMDBDataset(test_dataset, tokenizer, max_length=256)

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

---

## Part 4: Build Classification Model

In [ ]:
class BertClassifier(nn.Module):
    """
    BERT + Classification head for sentiment analysis.
    
    Architecture:
        BERT -> [CLS] embedding -> Dropout -> Linear -> Labels
    """
    
    def __init__(self, bert_model_name="bert-base-uncased", num_classes=2, dropout=0.1):
        super().__init__()
        
        # Load pre-trained BERT
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask):
        """
        Forward pass.
        
        Args:
            input_ids: Token IDs (batch, seq_len)
            attention_mask: Attention mask (batch, seq_len)
            
        Returns:
            logits: Class logits (batch, num_classes)
        """
        # Get BERT outputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Use pooler output ([CLS] token representation)
        pooled = outputs.pooler_output
        
        # Classification
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)
        
        return logits

# Create model
model = BertClassifier(num_classes=2)
model = model.to(device)

print(f"Model parameters:")
bert_params = sum(p.numel() for p in model.bert.parameters())
classifier_params = sum(p.numel() for p in model.classifier.parameters())
print(f"  BERT: {bert_params:,}")
print(f"  Classifier: {classifier_params:,}")
print(f"  Total: {bert_params + classifier_params:,}")

### Alternative: Using Hugging Face's Built-in

Hugging Face provides `BertForSequenceClassification` which does the same thing:

In [ ]:
# Alternatively, use the built-in class:
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased",
#     num_labels=2
# )

# We built our own to understand what's happening!

---

## Part 5: Training Setup

### Learning Rate Schedulers

When fine-tuning Transformers, we use learning rate schedulers with **warmup**:
1. Start with a small LR and gradually increase (warmup phase)
2. Then decrease LR over the rest of training

**Key scheduler methods:**
- `scheduler.step()`: Update the learning rate (call after each optimizer step)
- `scheduler.get_last_lr()`: Returns a list of current learning rates (useful for logging/visualization)

In [ ]:
# Training configuration
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5  # Small LR for fine-tuning (don't disturb BERT too much)
WARMUP_RATIO = 0.1

# Optimizer (AdamW is standard for Transformers)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

# Learning rate scheduler with warmup
total_steps = len(train_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Loss function
criterion = nn.CrossEntropyLoss()

print(f"Training configuration:")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")

In [ ]:
# Visualize learning rate schedule

lrs = []
temp_optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
temp_scheduler = get_linear_schedule_with_warmup(
    temp_optimizer, warmup_steps, total_steps
)

for _ in range(total_steps):
    lrs.append(temp_scheduler.get_last_lr()[0])
    temp_scheduler.step()

plt.figure(figsize=(10, 4))
plt.plot(lrs)
plt.xlabel('Training Step')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule with Warmup')
plt.axvline(x=warmup_steps, color='r', linestyle='--', label=f'Warmup ends ({warmup_steps} steps)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Why warmup? Starting with a smaller LR prevents early instability.")

---

## Part 6: Training Loop

In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, criterion, device):
    """
    Train for one epoch.
    
    Returns:
        Average loss for the epoch
    """
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for batch in progress_bar:
        # Move to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping (prevents exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        scheduler.step()
        
        # Track metrics
        total_loss += loss.item()
        predictions = logits.argmax(dim=-1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{100*correct/total:.1f}%"
        })
    
    return total_loss / len(dataloader), correct / total


def evaluate(model, dataloader, criterion, device):
    """
    Evaluate the model.
    
    Returns:
        Average loss and accuracy
    """
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            total_loss += loss.item()
            predictions = logits.argmax(dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return total_loss / len(dataloader), correct / total, all_predictions, all_labels

In [ ]:
# Training!
print("Starting training...")
print("=" * 50)

train_losses = []
train_accs = []
test_losses = []
test_accs = []

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, optimizer, scheduler, criterion, device
    )
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, _, _ = evaluate(
        model, test_loader, criterion, device
    )
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {100*train_acc:.1f}%")
    print(f"  Test Loss:  {test_loss:.4f}, Test Acc:  {100*test_acc:.1f}%")

total_time = time.time() - start_time
print(f"\n✅ Training complete in {total_time:.1f} seconds!")

In [ ]:
# Visualize training progress

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
ax = axes[0]
ax.plot(range(1, NUM_EPOCHS + 1), train_losses, 'b-o', label='Train')
ax.plot(range(1, NUM_EPOCHS + 1), test_losses, 'r-o', label='Test')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training and Test Loss')
ax.legend()
ax.grid(True, alpha=0.3)

# Accuracy
ax = axes[1]
ax.plot(range(1, NUM_EPOCHS + 1), [a*100 for a in train_accs], 'b-o', label='Train')
ax.plot(range(1, NUM_EPOCHS + 1), [a*100 for a in test_accs], 'r-o', label='Test')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Training and Test Accuracy')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final Test Accuracy: {100*test_accs[-1]:.1f}%")

---

## Part 7: Detailed Evaluation

In [ ]:
# Try to import sklearn for detailed metrics
HAS_SKLEARN = False
try:
    from sklearn.metrics import classification_report, confusion_matrix
    HAS_SKLEARN = True
except ImportError:
    print("⚠️ sklearn not installed. Install with: pip install scikit-learn")
    print("Continuing with basic accuracy metrics...")

# For visualization
HAS_SEABORN = False
try:
    import seaborn as sns
    HAS_SEABORN = True
except ImportError:
    print("⚠️ seaborn not installed for confusion matrix visualization")

# Get final predictions
_, _, predictions, labels = evaluate(model, test_loader, criterion, device)

if HAS_SKLEARN:
    # Classification report
    print("Classification Report:")
    print("=" * 50)
    print(classification_report(
        labels, predictions, 
        target_names=['Negative', 'Positive']
    ))

    # Confusion matrix
    cm = confusion_matrix(labels, predictions)

    plt.figure(figsize=(6, 5))
    if HAS_SEABORN:
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive']
        )
    else:
        plt.imshow(cm, cmap='Blues')
        plt.colorbar()
        # Add annotations
        for i in range(2):
            for j in range(2):
                plt.text(j, i, str(cm[i, j]), ha='center', va='center', fontsize=14)
        plt.xticks([0, 1], ['Negative', 'Positive'])
        plt.yticks([0, 1], ['Negative', 'Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()
else:
    # Manual accuracy calculation
    accuracy = sum(p == l for p, l in zip(predictions, labels)) / len(labels)
    print(f"\nTest Accuracy: {accuracy*100:.1f}%")
    
    # Manual confusion matrix
    tp = sum(p == 1 and l == 1 for p, l in zip(predictions, labels))
    tn = sum(p == 0 and l == 0 for p, l in zip(predictions, labels))
    fp = sum(p == 1 and l == 0 for p, l in zip(predictions, labels))
    fn = sum(p == 0 and l == 1 for p, l in zip(predictions, labels))
    
    print(f"\nConfusion Matrix:")
    print(f"  True Negatives:  {tn}")
    print(f"  False Positives: {fp}")
    print(f"  False Negatives: {fn}")
    print(f"  True Positives:  {tp}")

---

## Part 8: Test on Custom Examples

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    """
    Predict sentiment for a single text.
    
    Returns:
        Sentiment label and confidence
    """
    model.eval()
    
    # Tokenize
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = F.softmax(logits, dim=-1)
        prediction = logits.argmax(dim=-1).item()
        confidence = probs[0, prediction].item()
    
    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment, confidence


# Test on custom examples
test_reviews = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Terrible film. Waste of time and money. Don't bother watching.",
    "It was okay, nothing special but not bad either.",
    "The acting was superb and the plot kept me on the edge of my seat!",
    "Boring and predictable. I fell asleep halfway through.",
    "A masterpiece of modern cinema. Truly unforgettable.",
]

print("Custom Predictions:")
print("=" * 70)

for review in test_reviews:
    sentiment, confidence = predict_sentiment(review, model, tokenizer, device)
    emoji = "😃" if sentiment == "Positive" else "😞"
    print(f"\n{emoji} {sentiment} ({confidence*100:.1f}% confident)")
    print(f"   \"{review[:60]}...\"" if len(review) > 60 else f"   \"{review}\"")

---

## Part 9: Freeze BERT Layers (Optional Optimization)

In [ ]:
# Sometimes, we only want to train the classification head
# This is faster and works well with limited data

def freeze_bert_layers(model, freeze=True):
    """
    Freeze or unfreeze BERT layers.
    
    When frozen, only the classifier head is trained.
    """
    for param in model.bert.parameters():
        param.requires_grad = not freeze
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")

# Example: freeze BERT, only train classifier
print("Before freezing:")
freeze_bert_layers(model, freeze=False)

print("\nAfter freezing:")
freeze_bert_layers(model, freeze=True)

# Unfreeze for continued training
freeze_bert_layers(model, freeze=False)

---

## Try It Yourself: Exercises

### Exercise 1: Train on Full Dataset

With DGX Spark's memory, try training on the full IMDB dataset (25K samples).

In [ ]:
# YOUR CODE HERE
# 1. Use the full training set: dataset['train']
# 2. Adjust batch size and epochs
# 3. Compare results with the subset

### Exercise 2: Multi-class Classification

Adapt the code for a multi-class task like emotion detection (happy, sad, angry, etc.).

In [ ]:
# YOUR CODE HERE
# 1. Load a multi-class dataset: load_dataset("emotion")
# 2. Modify BertClassifier for num_classes > 2
# 3. Train and evaluate

---

## Common Mistakes

### Mistake 1: Too high learning rate

In [ ]:
# Wrong: Learning rate too high for fine-tuning
# optimizer = AdamW(model.parameters(), lr=1e-3)  # 0.001 is WAY too high!

# Right: Use small learning rate (2e-5 to 5e-5)
# optimizer = AdamW(model.parameters(), lr=2e-5)

print("Fine-tuning learning rates:")
print("  Too high (1e-3): Will destroy pre-trained knowledge!")
print("  Just right (2e-5 to 5e-5): Gently adapts BERT")
print("  Too low (1e-6): Training will be too slow")

### Mistake 2: Forgetting to set model to eval mode

In [ ]:
# Wrong: Evaluating in training mode
def wrong_evaluate(model, text):
    # model is still in train mode!
    # Dropout is active, results will be noisy
    pass

# Right: Set to eval mode
def right_evaluate(model, text):
    model.eval()  # Disables dropout!
    with torch.no_grad():  # Saves memory
        # ... prediction code
        pass

print("model.eval() disables dropout and batch norm updates.")
print("model.train() re-enables them for training.")

### Mistake 3: Not using gradient clipping

In [ ]:
# Transformers can have unstable gradients
# Always clip gradients during fine-tuning!

# In training loop:
# loss.backward()
# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Important!
# optimizer.step()

print("Gradient clipping prevents:")
print("  - Exploding gradients")
print("  - Training instability")
print("  - NaN losses")

---

## Checkpoint

You've learned:
- ✅ What BERT is and how it was pre-trained
- ✅ How to load and use pre-trained BERT
- ✅ Building a classification head on top of BERT
- ✅ Fine-tuning best practices (LR, warmup, gradient clipping)
- ✅ Evaluating with classification metrics
- ✅ Using predictions on new data

---

## Challenge (Optional)

Implement **Layer-wise Learning Rate Decay (LLRD)**:
- Lower learning rates for earlier BERT layers
- Higher learning rates for later layers and classifier
- This often improves fine-tuning results!

In [ ]:
# Challenge: Implement LLRD
# Hint: Create parameter groups with different LRs

# Example structure:
# param_groups = [
#     {'params': bert.encoder.layer[0].parameters(), 'lr': 1e-5},
#     {'params': bert.encoder.layer[1].parameters(), 'lr': 1.5e-5},
#     ...
#     {'params': classifier.parameters(), 'lr': 5e-5},
# ]

# YOUR CODE HERE

---

## Further Reading

- [BERT Paper](https://arxiv.org/abs/1810.04805) - Original BERT paper
- [Hugging Face Course](https://huggingface.co/learn/nlp-course) - Comprehensive tutorial
- [How to Fine-Tune BERT](https://mccormickml.com/2019/07/22/BERT-fine-tuning/) - Practical guide
- [RoBERTa](https://arxiv.org/abs/1907.11692) - Improved BERT training

---

## Cleanup

In [ ]:
# Clean up GPU memory
import gc

del model, bert, optimizer, scheduler
del train_loader, test_loader, train_ds, test_ds

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("Memory cleared! Ready for the next notebook.")

---

## Next Up

In **Notebook 06: GPT Text Generation**, we'll:
- Load a GPT-style language model
- Implement different decoding strategies
- Generate creative text with sampling and beam search

---

*Congratulations! You've fine-tuned your first BERT model! This is the same technique used in production ML systems worldwide.*